# Aplicación de una RNN para generar texto

Las RNN (Redes Neuronales Recurrentes) están diseñadas para trabajar con datos de secuencia.

Ejemplo de transformación de corpus a datos de entrenamiento para una RNN.

```python
corpus = "Quiero un jugo de naranja fresco"
seq_lenght = 3
```

|        x        |    y    |
|:---------------:|:-------:|
|  Quiero un jugo |    de   |
|    un jugo de   | naranja |
| jugo de naranja |  fresco |

#### Importar Librerias

In [34]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import re
import time
import numpy as np
import pandas as pd
import tensorflow as tf # https://www.tensorflow.org/install

Vamos a crear un generador de texto usando los discursos de Uribe

In [35]:
df = pd.read_csv("archivos/uribe.csv")
df.head()

,url,titulo,discurso,fecha,lugar
0,http://web.presidencia.gov.co/discursos/discur...,turistica_24022010,“Yo quiero felicitarlos d...,(Bogotá),Febrero 24 de 2010
1,http://web.presidencia.gov.co/discursos/discur...,lloreda_20012010,“Habríamos querido tener ...,(Bogotá),Enero 20 de 2009
2,http://web.presidencia.gov.co/discursos/discur...,bananeros_29042010,"Medellín, 29 abr (SP). “N...",(Medellín),29 de abril de 2010
3,http://web.presidencia.gov.co/discursos/discur...,ccg264_31012010,“Muy apreciados compatrio...,(Bucaramanga),Enero 31 de 2010
4,http://web.presidencia.gov.co/discursos/discur...,audiovisual_21012010,“Una ...,(Bogotá),Enero 21 de 2009


Corpus

In [36]:
text = df.discurso.str.cat(sep=" ")

# RNN

Primero necesitamos vectorizar el texto. Es decir, convertir el corpus en representación numérica.

Para esto creamos dos tablas de búsqueda: de caracteres a números (`char2idx`) y de números a caracteres (`idx2char`)

In [38]:
vocab = sorted(set(text))
print(f'{len(vocab):,.0f} caracteres únicos')

104 caracteres únicos


In [40]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [42]:
# Traducir nuestro texto a números
text_as_int = np.array([char2idx[c] for c in text])

In [43]:
# Ejemplo
np.array([char2idx[c] for c in "Qué chévere las redes neuronales"])

array([38, 68, 90,  0, 50, 55, 90, 69, 52, 65, 52,  0, 59, 48, 66,  0, 65,
       52, 51, 52, 66,  0, 61, 52, 68, 65, 62, 61, 48, 59, 52, 66])

#### La tarea de predicción
Dado un caracter, o una secuencia de caracteres, ¿cuál es el siguiente caracter más probable? 

#### Ejemplo

```python
seq_length = 4

Texto = "Hello"

Input = "Hell"
Output = "ello"
```

**Epoch**: Una epoca es la cantidad de pasos completos en el conjunto de datos de entrenamiento

In [47]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1) # división entera

In [48]:
# Crear el conjunto de datos de entrenamiento
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [49]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [51]:
split_input_target("Estas redes neuronales están muy densas")

('Estas redes neuronales están muy densa',
 'stas redes neuronales están muy densas')

In [52]:
dataset = sequences.map(split_input_target)

In [53]:
for i,(input_example, target_example) in  enumerate(dataset.take(1)):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '                    \xa0“Yo quiero felicitarlos de todo corazón por esta nueva  Vitrina Turística que t'
Target data: '                   \xa0“Yo quiero felicitarlos de todo corazón por esta nueva  Vitrina Turística que ta'


In [54]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print(f"Paso {i}")
    print(f"  input: {input_idx} ({repr(idx2char[input_idx])})")
    print(f"  expected output: {target_idx} ({repr(idx2char[target_idx])})")
    print()

Paso 0
  input: 0 (' ')
  expected output: 0 (' ')

Paso 1
  input: 0 (' ')
  expected output: 0 (' ')

Paso 2
  input: 0 (' ')
  expected output: 0 (' ')

Paso 3
  input: 0 (' ')
  expected output: 0 (' ')

Paso 4
  input: 0 (' ')
  expected output: 0 (' ')



#### Crear lotes de entrenamiento
Usaste `tf.data` para dividir el texto en secuencias manejables. Pero antes de introducir estos datos en el modelo, tienes que mezclar los datos y empaquetarlos en lotes.

**Batch size**: Tamaño del lote. El número de meustras antes de que el modelo sea actualizado.

**Buffer size**: Tamaño del buffer. Evita que se baraje todo el conjunto de datos.

In [55]:
BATCH_SIZE = 256

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((256, 100), (256, 100)), types: (tf.int64, tf.int64)>

#### Crear el modelo
Use `tf.keras.Sequential` para definir el modelo. Este modelo va a tener tres capas:

- `tf.keras.layers.Embedding`: Capa de entrada
- `tf.keras.layers.GRU`: RNN
- `tf.keras.layers.Dense`: Capa de salida

In [56]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 64

# Number of RNN units
rnn_units = 512

In [57]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

In [58]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (256, None, 64)           6656      
_________________________________________________________________
gru_4 (GRU)                  (256, None, 512)          887808    
_________________________________________________________________
gru_5 (GRU)                  (256, None, 512)          1575936   
_________________________________________________________________
dense_2 (Dense)              (256, None, 104)          53352     
Total params: 2,523,752
Trainable params: 2,523,752
Non-trainable params: 0
_________________________________________________________________


#### Ensayar el modelo

In [59]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(256, 100, 104) # (batch_size, sequence_length, vocab_size)


In [60]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (256, None, 64)           6656      
_________________________________________________________________
gru_4 (GRU)                  (256, None, 512)          887808    
_________________________________________________________________
gru_5 (GRU)                  (256, None, 512)          1575936   
_________________________________________________________________
dense_2 (Dense)              (256, None, 104)          53352     
Total params: 2,523,752
Trainable params: 2,523,752
Non-trainable params: 0
_________________________________________________________________


In [61]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [62]:
sampled_indices

array([ 26, 102,  29,  68,  10,  93,  79,  80,  84,   8,  27,  28,  15,
         8,  28,  79,  52,  47,  38,  66,  43,  63,  60,  22,  50,  56,
        10,  75,  63,  18,   4,  66,  45,  62,  51,  77,  50,  54,  55,
        13,  68,  45,  85,  17,  16,  38,  87,   9,  70,  66,  52,  55,
        19,   5,  28,  10,  43,  95,  82,  11,  16,  16,  80,  17,  81,
        19,  46,   9,  92,  68,  84,  91,  13,  68,  94,  55,  31,  26,
        20, 100,  80,  78,  25,  79,  63,  89,  65, 101,   0,  62,  63,
        83,  35,  81,  43,  95, 101,   9,  80,  90])

In [63]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' bien importante. El país ha venido avanzando en eso.                        Y me gusta mucho saber '

Next Char Predictions: 
 'E•Hu1ó´ºÍ/FG6/G´eZQsVpmAci1\xa0p9)sXod¬cgh4uXÓ87Qá0wseh:,G1VüÁ277º8¿:Y0ñuÍí4uúhJE;“º°D´pçr” opÉN¿Vü”0ºé'


# Basura

Como podemos observar, el modelo por ahora sólo produce basura. Por eso lo tenemos que entrenar.

In [64]:
# Función de perdida para optimizar el modelo
def loss(labels, logits):
      return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [65]:
model.compile(optimizer='adam', loss=loss)

Entrenar una red neuronal es demorado, por eso debemos ir guardando el progreso

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'training_checkpoints/'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_prefix,
        save_weights_only=True)

Entrenar el modelo
<center><img src='img/waiting.jpg' style='height:300px; float: center; margin: 0px 15px 15px 0px'></center>

In [66]:
# Una epoca es la cantidad de pasos completos en el conjunto de datos de entrenamiento
EPOCHS=10
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)) # Comentar esta linea la primera vez 
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
 46/209 [=====>........................] - ETA: 9:17 - loss: 0.8566

KeyboardInterrupt: 

Cargar modelo

In [67]:
p_model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
p_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
p_model.build(tf.TensorShape([1, None]))

Hacer predicciones

In [68]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return(start_string + ''.join(text_generated))

In [72]:
print(generate_text(p_model, start_string=u"Trabajar, trabajar y trabajar "))

Trabajar, trabajar y trabajar con toda   autoducción, los resultados sondiados en acelerada coberturas sociales.                        Y si los países                       Pero también, que era el crecimiento del Instituto Colombiano de Bienestar Familiar, al ex l. Por la compañero de Superintendencia   Diego Jojos.                        Ayer le dije, ¿cómo le va a tener en Colombia. Y que deso de que traigan ese monopolio.                       Y dar muy importante es el país un gobierno, como proporción escolar y la   independencia del mundo en estos años. El país le da a mí procedimía muy buques, a General de la Nación,   tiene que dar es que reciben los dos operaciones de todos mis compatriotas. Y a uno lo   sotros tenemos que ampliar reaminar un período de Gobierno:                        ¿Cuál es el camino para  desde aquellos   compañeros de los pobres.                        Pero tiene un 48 por ciento; está en  mar en Nariño. Nos supería la prensa especie de que todos son e